Data

... creating DataFrames

In [3]:
df_got_info = spark.sql("SELECT * FROM got_csv_laura2_csv")


In [4]:
df_got_sub = spark.sql("SELECT * FROM subtitle_list_laura_csv")

... creating RDDs

In [6]:
rdd_info = df_got_info.rdd.map(tuple)
rdd_sub = df_got_sub.rdd.map(tuple)

DOCUMENTS (episodes)

Inverted and non inverted index for documents (subtitles by episode)

... tokenizing

In [10]:
import os
import re

baseDir = os.path.join('databricks-datasets')
inputPath = os.path.join('cs100', 'lab3', 'data-001')
STOPWORDS_PATH = 'stopwords.txt'

stopfile = os.path.join(baseDir, inputPath, STOPWORDS_PATH)
stopwords = set(sc.textFile(stopfile).collect())
stopwords.add("re")
stopwords.add("ll")
stopwords.add("ve")
stopwords.add("m")
stopwords.add("s")
stopwords.add("t")
stopwords.add("don")
print ('These are the stopwords: %s' % stopwords)

split_regex = r'\W+'

def tokenize(string):
    """ An implementation of input string tokenization that excludes stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    terms = []
    if string != ' ':
      terms_all = re.split(split_regex, string.lower())
      for term in terms_all:
        if len(term) and (term not in stopwords):
          terms.append(term)
    return terms


These are the stopwords: {'are', 'as', 'my', 'should', 'into', 'in', 'about', 'if', 'again', 'very', 'here', 'this', 'be', 'she', 'up', 'is', 'we', 'myself', 'other', 'such', 'can', 'i', 'do', 'he', 'both', 'them', 'doing', 't', 'her', 'now', 'for', 'was', 'their', 'ourselves', 'there', 'herself', 'further', 'over', 'been', 'yourself', 'and', 'only', 'theirs', 'yours', 'any', 'to', 'why', 're', 'yourselves', 'same', 'most', 'below', 'what', 'off', 'nor', 'you', 'when', 'm', 'then', 'your', 'had', 'before', 'his', 'under', 'above', 'not', 'which', 'so', 'that', 'at', 'more', 'him', 'while', 'did', 'how', 'own', 'on', 'our', 'they', 'or', 'having', 'whom', 'the', 'have', 'were', 'of', 'between', 'just', 'who', 'will', 'all', 'me', 'by', 'itself', 'until', 've', 'too', 'few', 'out', 'those', 'against', 'himself', 'after', 'some', 'but', 'ours', 'through', 'than', 'a', 'it', 'these', 'from', 'no', 'themselves', 'with', 'during', 'hers', 'does', 'being', 'an', 'once', 'don', 'down', 'has', 'each', 'because', 'll', 'where', 'am', 's', 'its'}

In [11]:
rdd_sub_token = rdd_sub.map(lambda s: (s[0], tokenize(s[5]))).flatMapValues(lambda x: x)

... noninvindex

In [13]:
# noninvindex

rdd_sub_noninvindex = rdd_sub_token.map(lambda a: (a, 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[0][0],(x[0][1], x[1]))).groupByKey().sortByKey()
rdd_sub_noninvindex = rdd_sub_noninvindex.map(lambda x : (x[0], list(x[1])))

... invindex

In [15]:
# invindex

rdd_sub_invindex= rdd_sub_token.map(lambda a: (a, 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[0][1],(x[0][0], x[1]))).groupByKey().sortByKey()
rdd_sub_invindex = rdd_sub_invindex.map(lambda x : (x[0], list(x[1])))

Precalculate Inverse document frequency for tokens in documents

In [17]:
import math

rdd_sub_invindex_docf = rdd_sub_invindex.map(lambda x: (x[0], len(x[1]), x[1]))
numDoc_rdd_sub = rdd_sub_noninvindex.count() # Anzahl Episoden = Anzahl Dokumente = 67
rdd_sub_idf = rdd_sub_invindex_docf.map(lambda x: (x[0], math.log((1 + numDoc_rdd_sub) / (1 + x[1])))) 

Precalculating dNorm for documents

Ja, ich weiss, der folgende Code ist nicht sehr schön.

In [20]:
# Konsolidierung Unteritel (dabei Berechnung ff pro Dokument ) und join der idf zu jedem Wort -> Episodennummer, Tokens in Episode mit ff (lokales Mass) und idf (globales Mass)
rdd_sub_dnorm = rdd_sub_token.map(lambda a: (a, 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[0][1],(x[0][0], x[1]))).join(rdd_sub_idf).sortByKey()
# Berechnung (ff * idf)^2 = a^2
rdd_sub_dnorm = rdd_sub_dnorm.map(lambda x: (x[1][0][0], (x[0], x[1][0][1], x[1][1]) ) )
rdd_sub_dnorm = rdd_sub_dnorm.map(lambda x: (x[0], (x[1][0], (x[1][1] * x[1][2])**2)))
# Summierung über (ff * idf)^2 und Ziehen der Wurzel -> dNorm
rdd_sub_dnorm = rdd_sub_dnorm.map(lambda x: (x[0], x[1][1])).reduceByKey(lambda a,b: a+b)
rdd_sub_dnorm = rdd_sub_dnorm.map(lambda x: (x[0], math.sqrt(x[1]))).sortByKey()

QUERIES (episode summary)

Tokenizing

In [23]:
rdd_info_token = rdd_info.map(lambda s: (s[0], tokenize(s[10])))
rdd_info_token = rdd_info_token.flatMapValues(lambda x: x)

Queryindex

In [25]:
rdd_info_queryindex = rdd_info_token.map(lambda a: (a, 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[0][0],(x[0][1], x[1]))).groupByKey().sortByKey()
rdd_info_queryindex = rdd_info_queryindex.map(lambda x : (x[0], list(x[1])))

In [26]:
rdd_info_invqueryindex= rdd_info_token.map(lambda a: (a, 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[0][1],(x[0][0], x[1]))).groupByKey().sortByKey()
rdd_info_invqueryindex = rdd_info_invqueryindex.map(lambda x : (x[0], list(x[1])))

Adding inverse document frequency to idf-RDD where query-token not in documents (document frequency = 0)

In [28]:
print("The idf of words that occur in the queries but not in any documents (document frequency = 0) is: ", math.log((1 + numDoc_rdd_sub)))

The idf of words that occur in the queries but not in any documents (document frequency = 0) is: 4.219507705176107

In [29]:
# Wörter in Queries, die noch nicht in idf-RDD der Dokumente sind
#list_missing_words_idf = []

#for word in rdd_info_invqueryindex.map(lambda x: x[0]).collect():
#  if word not in rdd_sub_idf.map(lambda x: x[0]).collect():
#    list_missing_words_idf.append((word, 4.219507705176107))
#print(list_missing_words_idf)

In [30]:
# vereinigen idf-RDD der Dokumente mit den idf der Abfragen, welche noch nicht vorhanden (document frequency = 0)
rdd_info_idf = rdd_info_invqueryindex.map(lambda x: (x[0], 4.219507705176107))

In [31]:
rdd_idf = rdd_sub_idf.fullOuterJoin(rdd_info_idf).sortByKey()
rdd_idf = rdd_idf.map(lambda x: (x[0], x[1][0] if x[1][0] != None else x[1][1]))

Calculate query norm

In [33]:
rdd_info_qnorm = rdd_info_token.map(lambda a: (a, 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[0][1],(x[0][0], x[1]))).join(rdd_idf).sortByKey()
rdd_info_qnorm = rdd_info_qnorm.map(lambda x: (x[1][0][0], (x[0], x[1][0][1], x[1][1]) ) )
rdd_info_qnorm = rdd_info_qnorm.map(lambda x: (x[0], (x[1][0], (x[1][1] * x[1][2])**2)))
rdd_info_qnorm = rdd_info_qnorm.map(lambda x: (x[0], x[1][1])).reduceByKey(lambda a,b: a+b)
rdd_info_qnorm = rdd_info_qnorm.map(lambda x: (x[0], math.sqrt(x[1]))).sortByKey()

BERECHNUNG RSV(q, dj)

...accumulator

In [36]:
rdd_a = rdd_sub_token.map(lambda a: (a, 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[0][1],(x[0][0], x[1]))).join(rdd_sub_idf).sortByKey()
rdd_a = rdd_a.map(lambda x: (x[1][0][0], (x[0], x[1][0][1], x[1][1]) ) )
rdd_a = rdd_a.map(lambda x: (x[1][0], (x[0], x[1][1] * x[1][2]))).sortByKey()

In [37]:
rdd_b= rdd_info_token.map(lambda a: (a, 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[0][1],(x[0][0], x[1]))).join(rdd_idf).sortByKey()
rdd_b = rdd_b.map(lambda x: (x[1][0][0], (x[0], x[1][0][1], x[1][1]) ) )
rdd_b = rdd_b.map(lambda x: (x[1][0], (x[0], (x[1][1] * x[1][2])))).sortByKey()

In [38]:
rdd_a_b_joined = rdd_a.join(rdd_b)
rdd_acc = rdd_a_b_joined.map(lambda x: ((x[1][0][0], x[1][1][0]), x[1][0][1] * x[1][1][1])).reduceByKey(lambda a,b: a + b)
rdd_acc = rdd_acc.map(lambda x: (x[0], ('acc', x[1])))

In [39]:
rdd_acc_dnorm_joined = rdd_acc.map(lambda x: (x[0][0], (x[0][1], x[1]))).join(rdd_sub_dnorm)
rdd_acc_dnorm_joined = rdd_acc_dnorm_joined.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],('dnorm',x[1][1]))))
rdd_acc_dnorm_qnorm_joined = rdd_acc_dnorm_joined.join(rdd_info_qnorm)
rdd_acc_dnorm_qnorm_joined = rdd_acc_dnorm_qnorm_joined.map(lambda x:  ( (x[0],  x[1][0][0] ) ,  x[1][0][1]  ,  x[1][0][2]  , ( 'qnorm', x[1][1])     ) )

In [40]:
rdd_rsv = rdd_acc_dnorm_qnorm_joined.map(lambda x: (( x[0][0] ,  (-1) * x[1][1] * 1000 / x[2][1] * x[3][1] ), x[0][1]  ) ).sortByKey()    # acc * 1000 / dnorm * qnorm
rdd_rsv.collect()

Out[ 61 ]: 
[(('QS01E01', -2947.869011426398), 'S02E03'),
 (('QS01E01', -2873.460213587193), 'S01E05'),
 (('QS01E01', -2500.2880779143743), 'S01E01'),
 (('QS01E01', -2416.9859116918196), 'S01E04'),
 (('QS01E01', -2025.9686454856164), 'S04E08'),
 (('QS01E01', -1690.2278355817946), 'S01E07'),
 (('QS01E01', -1530.0863533903482), 'S04E06'),
 (('QS01E01', -1492.1253336670793), 'S01E08'),
 (('QS01E01', -1473.3530897663356), 'S07E02'),
 (('QS01E01', -1412.7270776859784), 'S05E08'),
 (('QS01E01', -1345.4100541808662), 'S04E07'),
 (('QS01E01', -1280.8609713596807), 'S06E09'),
 (('QS01E01', -1195.8659314805313), 'S02E04'),
 (('QS01E01', -1160.7338266730271), 'S07E03'),
 (('QS01E01', -1117.1289519744707), 'S05E01'),
 (('QS01E01', -1069.7583561586553), 'S03E05'),
 (('QS01E01', -1055.328534345791), 'S03E06'),
 (('QS01E01', -1053.2572278969892), 'S02E10'),
 (('QS01E01', -1011.882927134421), 'S05E06'),
 (('QS01E01', -1001.5730439892964), 'S01E03'),
 (('QS01E01', -993.8454021659923), 'S06E04'),
 (('QS01E01', -938.4277475550429), 'S02E07'),
 (('QS01E01', -934.6575281675317), 'S03E04'),
 (('QS01E01', -877.883625165537), 'S03E07'),
 (('QS01E01', -844.6341658179261), 'S07E07'),
 (('QS01E01', -840.9746724613337), 'S01E02'),
 (('QS01E01', -802.4661210827808), 'S01E06'),
 (('QS01E01', -795.1622861140957), 'S03E01'),
 (('QS01E01', -744.6648847078669), 'S05E07'),
 (('QS01E01', -724.5599275872868), 'S02E01'),
 (('QS01E01', -714.563605588762), 'S05E04'),
 (('QS01E01', -673.6967896539747), 'S06E05'),
 (('QS01E01', -671.5289920097348), 'S03E02'),
 (('QS01E01', -623.3284100276529), 'S02E08'),
 (('QS01E01', -609.2211622059662), 'S01E09'),
 (('QS01E01', -586.9781707279202), 'S07E01'),
 (('QS01E01', -563.377547190198), 'S03E03'),
 (('QS01E01', -523.1791290648832), 'S01E10'),
 (('QS01E01', -492.43202316371026), 'S04E05'),
 (('QS01E01', -455.67357800832264), 'S02E06'),
 (('QS01E01', -446.17182022360134), 'S06E07'),
 (('QS01E01', -424.18731098260207), 'S02E02'),
 (('QS01E01', -413.01802354703665), 'S07E04'),
 (('QS01E01', -411.9986321201718), 'S02E09'),
 (('QS01E01', -401.2469409986108), 'S03E08'),
 (('QS01E01', -349.1279441037233), 'S05E05'),
 (('QS01E01', -310.26019443155934), 'S04E02'),
 (('QS01E01', -307.1147095898928), 'S04E04'),
 (('QS01E01', -301.24569402378364), 'S05E10'),
 (('QS01E01', -300.1351527084706), 'S05E03'),
 (('QS01E01', -293.2279934814867), 'S02E05'),
 (('QS01E01', -285.9375835489952), 'S06E03'),
 (('QS01E01', -270.0048753258816), 'S03E09'),
 (('QS01E01', -245.19973684634442), 'S06E06'),
 (('QS01E01', -244.68512558816622), 'S03E10'),
 (('QS01E01', -230.80886692235313), 'S04E03'),
 (('QS01E01', -178.80658291088733), 'S06E02'),
 (('QS01E01', -171.4595260481709), 'S07E05'),
 (('QS01E01', -169.76115702023895), 'S05E02'),
 (('QS01E01', -160.75873022986778), 'S05E09'),
 (('QS01E01', -159.7565596705597), 'S07E06'),
 (('QS01E01', -147.90950079887847), 'S04E10'),
 (('QS01E01', -139.02310156946868), 'S06E10'),
 (('QS01E01', -118.31065993225349), 'S04E01'),
 (('QS01E01', -110.6214979855951), 'S04E09'),
 (('QS01E01', -109.0287794954337), 'S06E01'),
 (('QS01E01', -51.94013625941945), 'S06E08'),
 (('QS01E02', -1405.0019727068482), 'S02E01'),
 (('QS01E02', -1320.6067196744282), 'S01E01'),
 (('QS01E02', -1010.2198292719961), 'S06E05'),
 (('QS01E02', -950.8769421659118), 'S03E10'),
 (('QS01E02', -925.5747646371995), 'S01E02'),
 (('QS01E02', -890.4248003304742), 'S06E02'),
 (('QS01E02', -817.1660550125328), 'S01E03'),
 (('QS01E02', -799.2086711322337), 'S03E02'),
 (('QS01E02', -746.6284456268141), 'S07E07'),
 (('QS01E02', -681.8775591697447), 'S01E08'),
 (('QS01E02', -675.9438253808909), 'S01E10'),
 (('QS01E02', -663.6361294375769), 'S03E04'),
 (('QS01E02', -662.8893852172373), 'S04E04'),
 (('QS01E02', -653.9277916909102), 'S01E05'),
 (('QS01E02', -612.9689716379665), 'S01E07'),
 (('QS01E02', -595.1714638745811), 'S06E10'),
 (('QS01E02', -574.4576470748401), 'S03E06'),
 (('QS01E02', -555.4189777842688), 'S04E05'),
 (('QS01E02', -513.4739331438208), 'S07E03'),


Writing data

In [42]:
rdd_rsv_flat = rdd_rsv.map(lambda x: (x[0][0], x[0][1], x[1]))
df_rsv = sqlContext.createDataFrame(rdd_rsv_flat)
df_rsv.repartition(1).write.format("com.databricks.spark.csv").option("header", "false").save('/FileStore/output_leistungsnachweis2.csv')

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2433716733276608> in <module> () 
 1 rdd_rsv_flat = rdd_rsv . map ( lambda x : ( x [ 0 ] [ 0 ] , x [ 0 ] [ 1 ] , x [ 1 ] ) ) 
 2 df_rsv = sqlContext . createDataFrame ( rdd_rsv_flat ) 
 ----> 3 df_rsv . repartition ( 1 ) . write . format ( "com.databricks.spark.csv" ) . option ( "header" , "false" ) . save ( '/FileStore/output_leistungsnachweis2.csv' ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 734 self . _jwrite . save ( ) 
 735 else : 
 --> 736 self . _jwrite . save ( path ) 
 737 
 738 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : 'path dbfs:/FileStore/output_leistungsnachweis2.csv already exists.;'

Downloading data

In [44]:
display(dbutils.fs.ls("/FileStore/output_leistungsnachweis2.csv"))

path,name,size
dbfs:/FileStore/output_leistungsnachweis2.csv/_SUCCESS,_SUCCESS,0
dbfs:/FileStore/output_leistungsnachweis2.csv/_committed_3182027064363458271,_committed_3182027064363458271,114
dbfs:/FileStore/output_leistungsnachweis2.csv/_started_3182027064363458271,_started_3182027064363458271,0
dbfs:/FileStore/output_leistungsnachweis2.csv/part-00000-tid-3182027064363458271-114f53e2-59e2-4cc4-9b2c-3ce7f73a899b-1857-1-c000.csv,part-00000-tid-3182027064363458271-114f53e2-59e2-4cc4-9b2c-3ce7f73a899b-1857-1-c000.csv,154254


Downloading data requires entering the following into your browser:
https://<databricks-instance-name>/files/my-stuff/my-file.txt?o=######, wobei vorausgesetzt wird, dass das File im Verzeichnis FileStore abgelegt ist und "my-stuff" mit dem weiteren Verzeichnis (in meinem Fall "output_leistungsnachweis2.csv") und ###### mit dem Zahlen Code hinter ?o= in the deployment URL ersetzt werden muss

-> https://community.cloud.databricks.com/files/output_leistungsnachweis2.csv/part-00000-tid-3182027064363458271-114f53e2-59e2-4cc4-9b2c-3ce7f73a899b-1857-1-c000.csv?o=1634364218142529